In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
TIME_FORMAT = '%Y-%m-%d %H:%M:%S'

train_df = pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/train.csv')
train_df['date_time'] = pd.to_datetime(train_df['date_time'], format=TIME_FORMAT)

test_df = pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/test.csv')
test_datetime = test_df['date_time']
test_df['date_time'] = pd.to_datetime(test_df['date_time'], format=TIME_FORMAT)

train_df.head()

In [ ]:
x_df = train_df[[c for c in train_df.columns if not c.startswith('target_')]]
y_df = train_df[[c for c in train_df.columns if c.startswith('target_')]]
x_df.head()

In [ ]:
y_df.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt 

sns.set_theme(style="darkgrid")

plt.figure(figsize=(20, 6))
sns.lineplot(x="date_time", y="deg_C", data=x_df)
sns.lineplot(x="date_time", y="deg_C", data=test_df)
plt.legend(['Training', 'Testing'])
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt 

sns.set_theme(style="darkgrid")

plt.figure(figsize=(20, 6))
sns.lineplot(x="date_time", y="absolute_humidity", data=x_df)
sns.lineplot(x="date_time", y="absolute_humidity", data=test_df)
plt.legend(['Training', 'Testing'])
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt 

sns.set_theme(style="darkgrid")
# deg_C	relative_humidity	absolute_humidity	sensor_1	sensor_2	sensor_3	sensor_4	sensor_5
plt.figure(figsize=(24, 6))
plt.subplot(1, 4, 1), sns.histplot(x="deg_C", data=x_df)
plt.subplot(1, 4, 2), sns.histplot(x="relative_humidity", data=x_df)
plt.subplot(1, 4, 3), sns.histplot(x="absolute_humidity", data=x_df)
plt.subplot(1, 4, 4)
sns.histplot(x="sensor_1", data=x_df, bins=np.arange(0, 3000, 50), color='C1', alpha=.5)
sns.histplot(x="sensor_2", data=x_df, bins=np.arange(0, 3000, 50), color='C2', alpha=.5)
sns.histplot(x="sensor_3", data=x_df, bins=np.arange(0, 3000, 50), color='C3', alpha=.5)
sns.histplot(x="sensor_4", data=x_df, bins=np.arange(0, 3000, 50), color='C4', alpha=.5)
sns.histplot(x="sensor_5", data=x_df, bins=np.arange(0, 3000, 50), color='C5', alpha=.5)
plt.show()

In [ ]:
for column in x_df.columns[1:]:
    mean, std = x_df[column].mean(), x_df[column].std()
    x_df[column] = (x_df[column] - mean) / std
    test_df[column] = (test_df[column] - mean) / std

In [ ]:
test_df

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt 

sns.set_theme(style="darkgrid")
# deg_C	relative_humidity	absolute_humidity	sensor_1	sensor_2	sensor_3	sensor_4	sensor_5
plt.figure(figsize=(24, 6))
plt.subplot(1, 4, 1), sns.histplot(x="deg_C", data=x_df)
plt.subplot(1, 4, 2), sns.histplot(x="relative_humidity", data=x_df)
plt.subplot(1, 4, 3), sns.histplot(x="absolute_humidity", data=x_df)
plt.subplot(1, 4, 4)
sns.histplot(x="sensor_1", data=x_df, bins=np.linspace(-3, 3, 50), color='C1', alpha=.5)
sns.histplot(x="sensor_2", data=x_df, bins=np.linspace(-3, 3, 50), color='C2', alpha=.5)
sns.histplot(x="sensor_3", data=x_df, bins=np.linspace(-3, 3, 50), color='C3', alpha=.5)
sns.histplot(x="sensor_4", data=x_df, bins=np.linspace(-3, 3, 50), color='C4', alpha=.5)
sns.histplot(x="sensor_5", data=x_df, bins=np.linspace(-3, 3, 50), color='C5', alpha=.5)
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
x = x_df.iloc[:, 1:].to_numpy()
y = y_df.to_numpy()
x_train, x_val, y_train, y_val = train_test_split(x, y)
x_train.shape, y_train.shape, x_val.shape, y_val.shape

## Trying a single output regression (`target_carbon_monoxide`)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor()
rf.fit(x_train, y_train[:,0])

In [ ]:
# Scoring
from sklearn.metrics import mean_squared_error
y_pred = rf.predict(x_val) #y_val
mean_squared_error(y_pred, y_val[:, 0])

## Multiple output regression

In [ ]:
from sklearn.multioutput import MultiOutputRegressor
morf = MultiOutputRegressor(RandomForestRegressor())
morf.fit(x_train, y_train)

In [ ]:
y_pred = morf.predict(x_val)
for i in range(3):
    print(f'{mean_squared_error(y_pred[:, i], y_val[:, i])}')

# Submission

In [ ]:
x_test = test_df.iloc[:, 1:].to_numpy()
y_test_pred = morf.predict(x_test)


In [ ]:
submission_df = pd.DataFrame(y_test_pred)
submission_df.columns = y_df.columns
submission_df = pd.concat([test_df['date_time'], submission_df], axis=1)
submission_df.to_csv('solution2.csv', index=False)

In [ ]:
submission_df.date_time.dt.strftime('%Y-%m-%d %H:%M:%S')


In [ ]:
test_datetime